In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [3]:
import ee
import geemap
service_account = ' boba-account@boba-430314.iam.gserviceaccount.com '
credentials = ee.ServiceAccountCredentials(service_account,'C:/Users/arj26323/OneDrive - University of Georgia/Documents/GEE AUTH/boba-430314-a19be859bfca.json')

ee.Initialize(credentials)

print(geemap.__version__)

0.34.1


In [4]:
Map = geemap.Map(center=[31.539096,-81.422318], zoom=10)

##Adding every plot coordinate
allplots_fc = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/GA_allplots_NEW.csv'
fc_all = geemap.csv_to_ee(allplots_fc, latitude = "Latitude", longitude = "Longitude")

In [10]:
#Cloud masking
aoi = fc_all
start_date = '2017-01-01'
end_date = '2024-12-31'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

s2_sr_cld_col_eval = get_s2_sr_cld_col(aoi, start_date, end_date)

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
#     return img.addBands(is_cld_shdw)

    return img_cloud_shadow.addBands(is_cld_shdw)

In [11]:
def cloudmask(image):
    # Select bands
    clouds = image.select('clouds')
    shadows = image.select('shadows')
    dark_pixels = image.select('dark_pixels')

    # Create a combined mask where any of the bands equal 1
    combined_mask = clouds.eq(1).Or(shadows.eq(1)).Or(dark_pixels.eq(1))

    # Invert the mask (setting 1s to 0s and vice versa)
    inverted_mask = combined_mask.Not()

    # Apply the inverted mask to the image
    return image.updateMask(inverted_mask)

In [39]:
# def addSENTINELFLATS(image):
#     flats = ee.Image(0).expression(
#         '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
#             'SWIR': image.select('B8'),
#             'RED': image.select('B4'),
#             'GREEN': image.select('B3')
#         })
    
#     return image.addBands(flats.rename('flats'))

def addSENTINELFLATS(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(RED-SWIR)/(RED+SWIR) - 41.2*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1)))', {
            'SWIR': image.select('B11'),
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        })
    
    return image.addBands(flats.rename('flats'))


def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('imagedate').toInt())

In [40]:
gaS2 = s2_sr_cld_col_eval.map(add_cld_shdw_mask).map(cloudmask).filterBounds(fc_all).map(addSENTINELFLATS).select('flats')

ultra_col = gaS2

In [26]:
def rasterExtraction(image, fc_subset):
    return image.sampleRegions(
        collection=fc_subset,  # Use the subset of locations
        scale=10,
        tileScale=8  # Adjust tile scale for performance
    ).map(lambda f: f.set('date', image.date().format('YYYY-MM-dd')))

out_dir = os.path.expanduser('~/Documents/Xin data/Sentinel')

# Split locations into batches of 100
batch_size = 100
fc_list = fc_all.toList(fc_all.size())  # Convert to list
num_features = fc_all.size().getInfo()

# Calculate the number of batches without math.ceil()
num_batches = (num_features + batch_size - 1) // batch_size  # Equivalent to math.ceil(num_features / batch_size)

years = range(2019, 2025)

for year in years:
    for batch in range(num_batches):
        # Subset locations for this batch
        fc_subset = ee.FeatureCollection(fc_list.slice(batch * batch_size, (batch + 1) * batch_size))

        print(f"Processing {year}, Batch {batch + 1}/{num_batches}...")

        # Filter Landsat images for the year
        subset = ultra_col.filterDate(f"{year}-01-01", f"{year}-12-31")

        # Fix: Use a lambda function to pass `fc_subset` properly
        extracted_features = subset.map(lambda img: rasterExtraction(img, fc_subset)).flatten()

        fc_dict = extracted_features.getInfo()
        features = [feature['properties'] for feature in fc_dict['features']]
        df = pd.DataFrame(features)
        df['date'] = pd.to_datetime(df['date'])

        # Save file in the specified directory
        filename = os.path.join(out_dir, f"sentinelflats_{year}_batch{batch + 1}.csv")
        df.to_csv(filename, index=False)
        print(f"Saved {filename}")

print("All batches processed!")

Processing 2019, Batch 1/33...
Saved C:\Users\arj26323/Documents/Xin data/Sentinel\sentinelflats_2019_batch1.csv
Processing 2019, Batch 2/33...


KeyboardInterrupt: 

In [41]:
Map.addLayer(ultra_col.first(), {}, 'xxx')
# Map

Map(bottom=1710864.0, center=[31.468203549389166, -81.25445494623303], controls=(WidgetControl(options=['posit…